In [261]:
import warnings
import datetime
import pandas as pd
import numpy as np
import statsmodels.api as sm

warnings.filterwarnings('ignore')

In [262]:
# Reading CSV files
data = pd.read_csv('processed_data_with_treated2.csv')

# Converting a date_column column to a datetime type
data['CRASH DATE'] = pd.to_datetime(data['CRASH DATE'])

data['CRASH DATE'] = data['CRASH DATE'].dt.strftime('%Y-%m-%d')

In [263]:
data_dict = {}
for index, row in data.iterrows():
    date = row['CRASH DATE']
    intersection_id = row['intersection_id']
    avg_temperature = row['avg_temperature']
    precipitation_amount = row['precipitation_amount']
    snowfall_amount = row['snowfall_amount']

    # Use a tuple of day and intersection_id as the key in the dictionary
    key = (date, intersection_id)
    data_dict[key] = {
        'avg_temperature': avg_temperature,
        'precipitation_amount': precipitation_amount,
        'snowfall_amount': snowfall_amount
    }


# data frame with weather
df_with_weather = pd.DataFrame.from_dict(data_dict, orient='index')
df_with_weather.reset_index(inplace=True)
df_with_weather
df_with_weather.rename(columns={'level_0': 'CRASH DATE', 'level_1': 'intersection_id'}, inplace=True)

# # Reorder the columns as desired
# df_with_weather = df_with_weather[['CRASH DATE', 'intersection_id', 'avg_temperature', 'precipitation_amount', 'snowfall_amount']]

In [264]:
df_with_weather

,CRASH DATE,intersection_id,avg_temperature,precipitation_amount,snowfall_amount
0,2021-09-11,0.0,22.1,0.2,0.0
1,2021-09-11,1001.0,22.1,0.2,0.0
2,2021-12-14,2.0,5.8,0.0,0.0
3,2021-12-14,3.0,5.8,0.0,0.0
4,2021-12-14,4.0,5.8,0.0,0.0
...,...,...,...,...,...
1748280,2023-04-26,52983.0,12.1,0.2,0.0
1748281,2023-05-22,5429.0,18.1,0.0,0.0
1748282,2023-05-19,2859.0,15.6,0.0,0.0
1748283,2023-03-10,34749.0,5.3,6.4,0.0


In [265]:
condition1 = data['CRASH DATE'] < '2021-12-09'
condition2 = data['CRASH DATE'] >= '2021-12-09'

# filter
data = data[condition1 & (data['PRE_CRASH'] == 'Making Left Turn') |
                 condition2 & (data['CONTRIBUTING FACTOR VEHICLE 1'] == 'Turning Improperly')]
data

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,Peak_gust_wind_speed,atmospheric_pressure,sunshine_duration,PRE_CRASH,CRASH_DATE,is_possible_treatement,left_turn_treatement_date,is_treatement,intersection_id,treated
8,2021-12-14,16:50,QUEENS,11413.0,40.675884,-73.755770,"(40.675884, -73.75577)",SPRINGFIELD BOULEVARD,EAST GATE PLAZA,NaN,...,NaN,1032.0,NaN,NaN,NaN,True,2019-12-20,1.0,8.0,1.0
44,2022-04-24,13:10,NaN,NaN,40.679955,-73.974910,"(40.679955, -73.97491)",SAINT MARKS AVENUE,6 AVENUE,NaN,...,NaN,1024.5,NaN,NaN,NaN,True,2018-12-06,1.0,44.0,1.0
130,2022-03-26,1:58,BRONX,10472.0,40.827070,-73.870750,"(40.82707, -73.87075)",NaN,NaN,1670 WATSON AVENUE,...,NaN,999.0,NaN,NaN,NaN,True,2019-12-19,1.0,130.0,1.0
178,2022-03-22,8:20,BRONX,10467.0,40.871510,-73.870570,"(40.87151, -73.87057)",BURKE AVENUE,BRONX PARK EAST,NaN,...,NaN,1021.2,NaN,NaN,NaN,True,2019-12-13,1.0,178.0,1.0
239,2021-07-08,9:25,NaN,NaN,40.697315,-73.932274,"(40.697315, -73.932274)",BUSHWICK AVENUE,NaN,NaN,...,NaN,1014.7,NaN,Making Left Turn,07/08/2021,True,2017-08-08,1.0,239.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1906165,2023-04-14,10:50,NaN,NaN,40.814636,-73.920334,"(40.814636, -73.920334)",3 AVENUE,EAST 146 STREET,NaN,...,NaN,1011.5,NaN,NaN,NaN,True,2021-12-09,0.0,-1.0,1.0
1906228,2023-05-20,9:53,NaN,NaN,40.704876,-73.812790,"(40.704876, -73.81279)",143 STREET,NaN,NaN,...,NaN,1017.3,NaN,NaN,NaN,True,2021-12-09,0.0,-1.0,1.0
1906254,2023-05-07,14:30,BRONX,10463.0,40.873910,-73.909164,"(40.87391, -73.909164)",NaN,NaN,49 WEST 225 STREET,...,NaN,1015.3,NaN,NaN,NaN,True,2021-12-09,0.0,-1.0,1.0
1906432,2023-05-22,12:00,NaN,NaN,40.712775,-74.005973,"(40.7127753, -74.0059728)",34 ROAD,94 STREET,NaN,...,NaN,1022.0,NaN,NaN,NaN,True,2021-12-09,0.0,-1.0,1.0


In [266]:
TMP = data[data['is_treatement'] == 1]
TMP

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,Peak_gust_wind_speed,atmospheric_pressure,sunshine_duration,PRE_CRASH,CRASH_DATE,is_possible_treatement,left_turn_treatement_date,is_treatement,intersection_id,treated
8,2021-12-14,16:50,QUEENS,11413.0,40.675884,-73.755770,"(40.675884, -73.75577)",SPRINGFIELD BOULEVARD,EAST GATE PLAZA,NaN,...,NaN,1032.0,NaN,NaN,NaN,True,2019-12-20,1.0,8.0,1.0
44,2022-04-24,13:10,NaN,NaN,40.679955,-73.974910,"(40.679955, -73.97491)",SAINT MARKS AVENUE,6 AVENUE,NaN,...,NaN,1024.5,NaN,NaN,NaN,True,2018-12-06,1.0,44.0,1.0
130,2022-03-26,1:58,BRONX,10472.0,40.827070,-73.870750,"(40.82707, -73.87075)",NaN,NaN,1670 WATSON AVENUE,...,NaN,999.0,NaN,NaN,NaN,True,2019-12-19,1.0,130.0,1.0
178,2022-03-22,8:20,BRONX,10467.0,40.871510,-73.870570,"(40.87151, -73.87057)",BURKE AVENUE,BRONX PARK EAST,NaN,...,NaN,1021.2,NaN,NaN,NaN,True,2019-12-13,1.0,178.0,1.0
239,2021-07-08,9:25,NaN,NaN,40.697315,-73.932274,"(40.697315, -73.932274)",BUSHWICK AVENUE,NaN,NaN,...,NaN,1014.7,NaN,Making Left Turn,07/08/2021,True,2017-08-08,1.0,239.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1904621,2023-05-14,20:27,MANHATTAN,10025.0,40.795250,-73.973210,"(40.79525, -73.97321)",WEST 96 STREET,WEST END AVENUE,NaN,...,NaN,1021.5,NaN,NaN,NaN,True,2018-09-13 00:00:00,1.0,90.0,1.0
1905637,2023-05-07,14:31,MANHATTAN,10027.0,40.808820,-73.948000,"(40.80882, -73.948)",NaN,NaN,132 WEST 125 STREET,...,NaN,1015.3,NaN,NaN,NaN,True,2021-06-17 00:00:00,1.0,162.0,1.0
1905687,2023-05-18,6:45,NaN,NaN,40.701836,-73.822110,"(40.701836, -73.82211)",JAMAICA AVENUE,130 STREET,NaN,...,NaN,1026.1,NaN,NaN,NaN,True,2019-12-19 00:00:00,1.0,864.0,1.0
1905721,2023-05-18,18:25,BROOKLYN,11213.0,40.679787,-73.938440,"(40.679787, -73.93844)",FULTON STREET,ALBANY AVENUE,NaN,...,NaN,1026.1,NaN,NaN,NaN,True,2018-04-12 00:00:00,1.0,585.0,1.0


In [267]:
# Groups by specified columns and calculates counts for each group
grouped_data = data.groupby(['CRASH DATE', 'treated', 'is_treatement', 'intersection_id', 'BOROUGH']).size().reset_index(name='crash number')

grouped_data

,CRASH DATE,treated,is_treatement,intersection_id,BOROUGH,crash number
0,2012-09-12,0.0,0.0,-1.0,QUEENS,1
1,2012-11-12,0.0,0.0,-1.0,MANHATTAN,1
2,2012-12-03,0.0,0.0,-1.0,BRONX COUNTY,1
3,2013-05-18,0.0,0.0,-1.0,BROOKLYN,1
4,2013-08-19,0.0,0.0,-1.0,QUEENS,1
...,...,...,...,...,...,...
26546,2023-06-01,1.0,0.0,-1.0,QUEENS,3
26547,2023-06-02,1.0,0.0,-1.0,BRONX,3
26548,2023-06-02,1.0,0.0,-1.0,BRONX COUNTY,1
26549,2023-06-02,1.0,0.0,-1.0,BROOKLYN,1


In [268]:
is_treatement = grouped_data[grouped_data['is_treatement'] == 1]
is_treatement

,CRASH DATE,treated,is_treatement,intersection_id,BOROUGH,crash number
13,2014-03-27,0.0,1.0,446.0,BROOKLYN,1
22,2014-04-25,0.0,1.0,732.0,BROOKLYN,1
24,2014-04-26,0.0,1.0,105.0,QUEENS,1
25,2014-04-26,0.0,1.0,240.0,STATEN ISLAND,1
35,2014-04-28,0.0,1.0,224.0,QUEENS,1
...,...,...,...,...,...,...
26507,2023-05-19,1.0,1.0,695.0,BROOKLYN,1
26511,2023-05-20,1.0,1.0,90.0,MANHATTAN,1
26518,2023-05-23,1.0,1.0,657.0,BROOKLYN,1
26535,2023-05-29,1.0,1.0,89.0,MANHATTAN,1


In [269]:
no_treatement = grouped_data[grouped_data['is_treatement'] == 0]
no_treatement

,CRASH DATE,treated,is_treatement,intersection_id,BOROUGH,crash number
0,2012-09-12,0.0,0.0,-1.0,QUEENS,1
1,2012-11-12,0.0,0.0,-1.0,MANHATTAN,1
2,2012-12-03,0.0,0.0,-1.0,BRONX COUNTY,1
3,2013-05-18,0.0,0.0,-1.0,BROOKLYN,1
4,2013-08-19,0.0,0.0,-1.0,QUEENS,1
...,...,...,...,...,...,...
26546,2023-06-01,1.0,0.0,-1.0,QUEENS,3
26547,2023-06-02,1.0,0.0,-1.0,BRONX,3
26548,2023-06-02,1.0,0.0,-1.0,BRONX COUNTY,1
26549,2023-06-02,1.0,0.0,-1.0,BROOKLYN,1


In [270]:
intersection_id_of1 = pd.unique(is_treatement['intersection_id'])
intersection_id_of1

array([446., 732., 105., 240., 224., 399., 502., 702., 855., 176., 359.,
       386., 396., 782., 117., 164., 246., 300., 378., 640., 820., 133.,
       182., 769., 259., 379., 685., 385., 624., 147., 297., 395., 722.,
        74., 589., 809.,  77., 501., 573., 579., 765., 836., 233., 394.,
       499., 549., 834., 122., 152., 231., 585., 688., 777., 444., 681.,
       438., 454., 704.,  85., 149., 315., 368., 414., 616., 137., 201.,
       729.,  67., 391., 531., 154., 524., 578., 760., 230., 293., 643.,
       703.,  53., 277., 309., 405., 518., 155., 184., 302., 854., 226.,
       292., 304., 320., 358., 461., 162., 316., 351., 519., 557., 678.,
        89., 409., 831., 363., 364., 728., 776., 581., 599., 129., 377.,
       420., 449., 552., 779.,  88., 311.,  57., 283., 329., 347., 606.,
        51., 125., 326., 747., 135., 209., 256., 521., 582., 818., 175.,
       322., 610., 825., 238., 830., 291., 670., 392., 279., 644., 720.,
       792.,  63., 575., 136., 284., 342., 462., 48

In [271]:
intersection_id_of0 = pd.unique(no_treatement['intersection_id'])
intersection_id_of0

array([-1.])

In [272]:
intersection_id_of1.size

754

In [273]:
choose_intersection_id_of0 = np.random.choice(intersection_id_of0, size=intersection_id_of1.size)
choose_intersection_id_of0

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1

In [274]:
specified_intersection_ids = np.concatenate((choose_intersection_id_of0, intersection_id_of1))
specified_intersection_ids

array([ -1.,  -1.,  -1., ...,   8., 821., 475.])

In [275]:
original_selected_data = data[data['intersection_id'].isin(specified_intersection_ids)]
original_selected_data

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,Peak_gust_wind_speed,atmospheric_pressure,sunshine_duration,PRE_CRASH,CRASH_DATE,is_possible_treatement,left_turn_treatement_date,is_treatement,intersection_id,treated
8,2021-12-14,16:50,QUEENS,11413.0,40.675884,-73.755770,"(40.675884, -73.75577)",SPRINGFIELD BOULEVARD,EAST GATE PLAZA,NaN,...,NaN,1032.0,NaN,NaN,NaN,True,2019-12-20,1.0,8.0,1.0
130,2022-03-26,1:58,BRONX,10472.0,40.827070,-73.870750,"(40.82707, -73.87075)",NaN,NaN,1670 WATSON AVENUE,...,NaN,999.0,NaN,NaN,NaN,True,2019-12-19,1.0,130.0,1.0
178,2022-03-22,8:20,BRONX,10467.0,40.871510,-73.870570,"(40.87151, -73.87057)",BURKE AVENUE,BRONX PARK EAST,NaN,...,NaN,1021.2,NaN,NaN,NaN,True,2019-12-13,1.0,178.0,1.0
250,2021-09-10,17:00,QUEENS,11372.0,40.748962,-73.891760,"(40.748962, -73.89176)",74 STREET,37 AVENUE,NaN,...,NaN,1012.3,NaN,Making Left Turn,09/10/2021,True,2016-06-30,1.0,250.0,1.0
257,2021-09-10,10:20,QUEENS,11419.0,40.692127,-73.834840,"(40.692127, -73.83484)",ATLANTIC AVENUE,111 STREET,NaN,...,NaN,1012.3,NaN,Making Left Turn,09/10/2021,True,2019-12-10,1.0,257.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1906165,2023-04-14,10:50,NaN,NaN,40.814636,-73.920334,"(40.814636, -73.920334)",3 AVENUE,EAST 146 STREET,NaN,...,NaN,1011.5,NaN,NaN,NaN,True,2021-12-09,0.0,-1.0,1.0
1906228,2023-05-20,9:53,NaN,NaN,40.704876,-73.812790,"(40.704876, -73.81279)",143 STREET,NaN,NaN,...,NaN,1017.3,NaN,NaN,NaN,True,2021-12-09,0.0,-1.0,1.0
1906254,2023-05-07,14:30,BRONX,10463.0,40.873910,-73.909164,"(40.87391, -73.909164)",NaN,NaN,49 WEST 225 STREET,...,NaN,1015.3,NaN,NaN,NaN,True,2021-12-09,0.0,-1.0,1.0
1906432,2023-05-22,12:00,NaN,NaN,40.712775,-74.005973,"(40.7127753, -74.0059728)",34 ROAD,94 STREET,NaN,...,NaN,1022.0,NaN,NaN,NaN,True,2021-12-09,0.0,-1.0,1.0


In [276]:
selected_treatement = grouped_data[grouped_data['intersection_id'].isin(specified_intersection_ids)]
selected_treatement

,CRASH DATE,treated,is_treatement,intersection_id,BOROUGH,crash number
0,2012-09-12,0.0,0.0,-1.0,QUEENS,1
1,2012-11-12,0.0,0.0,-1.0,MANHATTAN,1
2,2012-12-03,0.0,0.0,-1.0,BRONX COUNTY,1
3,2013-05-18,0.0,0.0,-1.0,BROOKLYN,1
4,2013-08-19,0.0,0.0,-1.0,QUEENS,1
...,...,...,...,...,...,...
26546,2023-06-01,1.0,0.0,-1.0,QUEENS,3
26547,2023-06-02,1.0,0.0,-1.0,BRONX,3
26548,2023-06-02,1.0,0.0,-1.0,BRONX COUNTY,1
26549,2023-06-02,1.0,0.0,-1.0,BROOKLYN,1


In [277]:
tmp = selected_treatement[selected_treatement['crash number'] > 1]
tmp

,CRASH DATE,treated,is_treatement,intersection_id,BOROUGH,crash number
26,2014-04-27,0.0,0.0,-1.0,BRONX,3
28,2014-04-27,0.0,0.0,-1.0,QUEENS,3
29,2014-04-28,0.0,0.0,-1.0,BRONX,3
30,2014-04-28,0.0,0.0,-1.0,BROOKLYN,9
31,2014-04-28,0.0,0.0,-1.0,MANHATTAN,5
...,...,...,...,...,...,...
26537,2023-05-30,1.0,0.0,-1.0,BROOKLYN,2
26541,2023-05-31,1.0,0.0,-1.0,BROOKLYN,3
26542,2023-05-31,1.0,0.0,-1.0,QUEENS,2
26546,2023-06-01,1.0,0.0,-1.0,QUEENS,3


In [278]:
# Construct date ranges, counted by each day, to compensate for dates when no incidents occurred (crash number recorded as 0)
start_date = datetime.datetime(2012, 7, 2)
end_date = datetime.datetime(2023, 6, 3)
date_range = pd.date_range(start_date, end_date, freq='D')
date_range

DatetimeIndex(['2012-07-02', '2012-07-03', '2012-07-04', '2012-07-05',
               '2012-07-06', '2012-07-07', '2012-07-08', '2012-07-09',
               '2012-07-10', '2012-07-11',
               ...
               '2023-05-25', '2023-05-26', '2023-05-27', '2023-05-28',
               '2023-05-29', '2023-05-30', '2023-05-31', '2023-06-01',
               '2023-06-02', '2023-06-03'],
              dtype='datetime64[ns]', length=3989, freq='D')

In [279]:
# Combination of generation date and intersection_id
all_combinations = pd.MultiIndex.from_product([date_range, specified_intersection_ids], names=['CRASH DATE', 'intersection_id'])
all_combinations = pd.DataFrame(index=all_combinations).reset_index()
all_combinations


,CRASH DATE,intersection_id
0,2012-07-02,-1.0
1,2012-07-02,-1.0
2,2012-07-02,-1.0
3,2012-07-02,-1.0
4,2012-07-02,-1.0
...,...,...
6015407,2023-06-03,860.0
6015408,2023-06-03,852.0
6015409,2023-06-03,8.0
6015410,2023-06-03,821.0


In [280]:
# Create a dictionary to record is_treatement and left_turn_treatement_date for each intersection_id
intersection_info = {}
for intersection_id in specified_intersection_ids:
    intersection_data = original_selected_data[original_selected_data['intersection_id'] == intersection_id].iloc[0]
    intersection_info[intersection_id] = {
        'is_treatement': intersection_data['is_treatement'],
        'left_turn_treatement_date': intersection_data['left_turn_treatement_date'],
        'BOROUGH': intersection_data['BOROUGH']
    }
intersection_info

{-1.0: {'is_treatement': 0.0,
  'left_turn_treatement_date': '2021-12-09',
  'BOROUGH': 'MANHATTAN'},
 446.0: {'is_treatement': 1.0,
  'left_turn_treatement_date': '2019-12-11 00:00:00',
  'BOROUGH': 'BROOKLYN'},
 732.0: {'is_treatement': 1.0,
  'left_turn_treatement_date': '2022-09-29 00:00:00',
  'BOROUGH': 'BROOKLYN'},
 105.0: {'is_treatement': 1.0,
  'left_turn_treatement_date': '2017-09-21 00:00:00',
  'BOROUGH': 'QUEENS'},
 240.0: {'is_treatement': 1.0,
  'left_turn_treatement_date': '2018-12-06 00:00:00',
  'BOROUGH': 'STATEN ISLAND'},
 224.0: {'is_treatement': 1.0,
  'left_turn_treatement_date': '2018-12-18 00:00:00',
  'BOROUGH': nan},
 399.0: {'is_treatement': 1.0,
  'left_turn_treatement_date': '2016-11-22 00:00:00',
  'BOROUGH': 'MANHATTAN'},
 502.0: {'is_treatement': 1.0,
  'left_turn_treatement_date': '2016-06-28 00:00:00',
  'BOROUGH': 'NEW YORK COUNTY'},
 702.0: {'is_treatement': 1.0,
  'left_turn_treatement_date': '2022-08-26 00:00:00',
  'BOROUGH': nan},
 855.0: {'is_

In [281]:
# Completing is_treatement and setting the value of treated
all_combinations['is_treatement'] = all_combinations['intersection_id'].map(lambda x: intersection_info[x]['is_treatement'])
all_combinations['left_turn_treatement_date'] = all_combinations['intersection_id'].map(lambda x: intersection_info[x]['left_turn_treatement_date'])
all_combinations['BOROUGH'] = all_combinations['intersection_id'].map(lambda x: intersection_info[x]['BOROUGH'])

# convert to datetime
all_combinations['left_turn_treatement_date'] = pd.to_datetime(all_combinations['left_turn_treatement_date'])

all_combinations['treated'] = all_combinations.apply(lambda row: 0 if row['CRASH DATE'] < row['left_turn_treatement_date'] else 1, axis=1)


In [282]:
# Fill in the missing "crash number" value to 0.
all_combinations = all_combinations[['CRASH DATE', 'treated', 'is_treatement', 'intersection_id', 'BOROUGH']]
all_combinations

,CRASH DATE,treated,is_treatement,intersection_id,BOROUGH
0,2012-07-02,0,0.0,-1.0,MANHATTAN
1,2012-07-02,0,0.0,-1.0,MANHATTAN
2,2012-07-02,0,0.0,-1.0,MANHATTAN
3,2012-07-02,0,0.0,-1.0,MANHATTAN
4,2012-07-02,0,0.0,-1.0,MANHATTAN
...,...,...,...,...,...
6015407,2023-06-03,1,1.0,860.0,BROOKLYN
6015408,2023-06-03,1,1.0,852.0,QUEENS
6015409,2023-06-03,1,1.0,8.0,QUEENS
6015410,2023-06-03,1,1.0,821.0,QUEENS


In [283]:
# Converting a "CRASH DATE" column to a datetime type
all_combinations['CRASH DATE'] = pd.to_datetime(all_combinations['CRASH DATE'])

# Formatted in year-month-day format
all_combinations['CRASH DATE'] = all_combinations['CRASH DATE'].dt.strftime('%Y-%m-%d')

# merge data
combined_df = pd.merge(all_combinations, selected_treatement[['CRASH DATE', 'intersection_id', 'crash number']], on=['CRASH DATE', 'intersection_id'], how='left')

# Fill in the missing "crash number" value to 0.
combined_df['crash number'] = combined_df['crash number'].fillna(0)

combined_df = pd.merge(combined_df, df_with_weather, on=['CRASH DATE', 'intersection_id'], how='left')

In [284]:
combined_df

,CRASH DATE,treated,is_treatement,intersection_id,BOROUGH,crash number,avg_temperature,precipitation_amount,snowfall_amount
0,2012-07-02,0,0.0,-1.0,MANHATTAN,0.0,NaN,NaN,NaN
1,2012-07-02,0,0.0,-1.0,MANHATTAN,0.0,NaN,NaN,NaN
2,2012-07-02,0,0.0,-1.0,MANHATTAN,0.0,NaN,NaN,NaN
3,2012-07-02,0,0.0,-1.0,MANHATTAN,0.0,NaN,NaN,NaN
4,2012-07-02,0,0.0,-1.0,MANHATTAN,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
17467161,2023-06-03,1,1.0,860.0,BROOKLYN,0.0,NaN,NaN,NaN
17467162,2023-06-03,1,1.0,852.0,QUEENS,0.0,NaN,NaN,NaN
17467163,2023-06-03,1,1.0,8.0,QUEENS,0.0,NaN,NaN,NaN
17467164,2023-06-03,1,1.0,821.0,QUEENS,0.0,NaN,NaN,NaN


In [285]:
combined_df = combined_df[combined_df['crash number'] > 0]

combined_df

,CRASH DATE,treated,is_treatement,intersection_id,BOROUGH,crash number,avg_temperature,precipitation_amount,snowfall_amount
108576,2012-09-12,0,0.0,-1.0,MANHATTAN,1.0,20.5,0.0,0.0
108577,2012-09-12,0,0.0,-1.0,MANHATTAN,1.0,20.5,0.0,0.0
108578,2012-09-12,0,0.0,-1.0,MANHATTAN,1.0,20.5,0.0,0.0
108579,2012-09-12,0,0.0,-1.0,MANHATTAN,1.0,20.5,0.0,0.0
108580,2012-09-12,0,0.0,-1.0,MANHATTAN,1.0,20.5,0.0,0.0
...,...,...,...,...,...,...,...,...,...
17464899,2023-06-02,1,0.0,-1.0,MANHATTAN,1.0,23.0,0.0,0.0
17464900,2023-06-02,1,0.0,-1.0,MANHATTAN,3.0,23.0,0.0,0.0
17464901,2023-06-02,1,0.0,-1.0,MANHATTAN,1.0,23.0,0.0,0.0
17464902,2023-06-02,1,0.0,-1.0,MANHATTAN,1.0,23.0,0.0,0.0


In [286]:
# Date of encoding
# combined_df['CRASH DATE'] = combined_df['CRASH DATE'].astype('datetime64').rank(method='dense').astype(int)
#
# combined_df = combined_df.sort_values(by='intersection_id', ascending=True)
#
# combined_df


In [290]:
combined_df['interaction_term'] = combined_df['treated'] * combined_df['is_treatement']

In [296]:
# Calculate the weight of each category
category_weights = combined_df['interaction_term'].value_counts().max() / combined_df['interaction_term'].value_counts()

# Creating a WLS (Weighted Least Squares) model using weights
weights = combined_df['interaction_term'].map(category_weights)



In [295]:
tmp = combined_df[combined_df['interaction_term'] == 1]
tmp

,CRASH DATE,treated,is_treatement,intersection_id,BOROUGH,crash number,avg_temperature,precipitation_amount,snowfall_amount,interaction_term
6024077,2016-07-03,1,1.0,434.0,BRONX,1.0,23.2,0.0,0.0,1.0
6122657,2016-07-21,1,1.0,250.0,QUEENS,1.0,26.3,0.0,0.0,1.0
6136896,2016-07-24,1,1.0,320.0,MANHATTAN,1.0,29.1,0.0,0.0,1.0
6142874,2016-07-25,1,1.0,77.0,BROOKLYN,1.0,27.0,46.7,0.0,1.0
6203629,2016-08-05,1,1.0,346.0,QUEENS,1.0,25.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
17424719,2023-05-19,1,1.0,695.0,NaN,1.0,15.6,0.0,0.0,1.0
17427650,2023-05-20,1,1.0,90.0,NaN,1.0,15.9,31.4,0.0,1.0
17435113,2023-05-23,1,1.0,657.0,BROOKLYN,1.0,15.9,0.0,0.0,1.0
17451431,2023-05-29,1,1.0,89.0,MANHATTAN,1.0,19.8,0.0,0.0,1.0


In [302]:

# columns = ['treated', 'is_treatement', 'avg_temperature', 'precipitation_amount', 'snowfall_amount', 'BOROUGH', 'interaction_term']
columns = ['interaction_term']
X = combined_df[columns]

X = pd.get_dummies(X)
y = combined_df['crash number']

In [303]:
model = sm.WLS(y, sm.add_constant(X), weights=weights)

# fit a model
results = model.fit()

In [304]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:           crash number   R-squared:                       0.273
Model:                            WLS   Adj. R-squared:                  0.273
Method:                 Least Squares   F-statistic:                 5.240e+06
Date:                Tue, 01 Aug 2023   Prob (F-statistic):               0.00
Time:                        16:45:03   Log-Likelihood:            -3.5999e+07
No. Observations:            13972111   AIC:                         7.200e+07
Df Residuals:                13972109   BIC:                         7.200e+07
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                3.7628      0.001   4417.229      0.000       3.761       3.764
interaction_term    -2.7576      0.001  -2289.051      0.000      -2.760      -2.755
==============================================================================
Omnibus:                  3465580.301   Durbin-Watson:                   2.051
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          7827927.452
Skew:                           1.420   Prob(JB):                         0.00
Kurtosis:                       5.319   Cond. No.                         2.62
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [307]:
columns = ['treated', 'is_treatement', 'avg_temperature', 'precipitation_amount', 'snowfall_amount', 'BOROUGH']
X = combined_df[columns]

X = pd.get_dummies(X)

In [308]:
model = sm.WLS(y, sm.add_constant(X), weights=weights)

# fit a model
results = model.fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:           crash number   R-squared:                       0.306
Model:                            WLS   Adj. R-squared:                  0.306
Method:                 Least Squares   F-statistic:                 4.735e+05
Date:                Tue, 01 Aug 2023   Prob (F-statistic):               0.00
Time:                        16:47:10   Log-Likelihood:            -3.5674e+07
No. Observations:            13972111   AIC:                         7.135e+07
Df Residuals:                13972097   BIC:                         7.135e+07
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                       4.0273      0.003   1356.199      0.000       4.021       4.033
treated                    -2.4370      0.003   -798.834      0.000      -2.443      -2.431
is_treatement              -0.5302      0.003   -162.374      0.000      -0.537      -0.524
avg_temperature            -0.0063   6.66e-05    -94.827      0.000      -0.006      -0.006
precipitation_amount        0.0084   6.38e-05    131.433      0.000       0.008       0.009
snowfall_amount            -0.0013   2.01e-05    -67.197      0.000      -0.001      -0.001
BOROUGH_BRONX               0.0068      0.004      1.891      0.059      -0.000       0.014
BOROUGH_BROOKLYN           -0.0034      0.003     -1.192      0.233      -0.009       0.002
BOROUGH_KINGS COUNTY       -2.4311      0.984     -2.471      0.013      -4.359      -0.503
BOROUGH_MANHATTAN           0.0029      0.003      1.097      0.273      -0.002       0.008
BOROUGH_NEW YORK COUNTY     0.0134      0.011      1.199      0.231      -0.009       0.035
BOROUGH_QUEENS             -0.0053      0.003     -1.861      0.063      -0.011       0.000
BOROUGH_QUEENS COUNTY       0.0776      0.035      2.238      0.025       0.010       0.146
BOROUGH_STATEN ISLAND      -0.0123      0.008     -1.624      0.104      -0.027       0.003
==============================================================================
Omnibus:                  3325582.068   Durbin-Watson:                   2.150
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          7630946.163
Skew:                           1.352   Prob(JB):                         0.00
Kurtosis:                       5.407   Cond. No.                     5.17e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.17e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""